# **Homework 2-1 Phoneme Classification**

## The DARPA TIMIT Acoustic-Phonetic Continuous Speech Corpus (TIMIT)
The TIMIT corpus of reading speech has been designed to provide speech data for the acquisition of acoustic-phonetic knowledge and for the development and evaluation of automatic speech recognition systems.

This homework is a multiclass classification task, 
we are going to train a deep neural network classifier to predict the phonemes for each frame from the speech corpus TIMIT.

link: https://academictorrents.com/details/34e2b78745138186976cbc27939b1b34d18bd5b3

## Download Data
Download data from google drive, then unzip it.

You should have `timit_11/train_11.npy`, `timit_11/train_label_11.npy`, and `timit_11/test_11.npy` after running this block.<br><br>
`timit_11/`
- `train_11.npy`: training data<br>
- `train_label_11.npy`: training label<br>
- `test_11.npy`:  testing data<br><br>

**notes: if the google drive link is dead, you can download the data directly from Kaggle and upload it to the workspace**




In [ ]:
# !gdown --id '1HPkcmQmFGu-3OknddKIa5dNDsR05lIQR' --output data.zip
!gdown --id '1IY-AUlTceDOXCuf9ZSJ_qZRQ8LzBtJUm' --output data.zip
!unzip data.zip
!ls 

Downloading...
From: https://drive.google.com/uc?id=1IY-AUlTceDOXCuf9ZSJ_qZRQ8LzBtJUm
To: /content/data.zip
372MB [00:04, 85.4MB/s]
Archive:  data.zip
   creating: timit_11/
  inflating: __MACOSX/._timit_11     
  inflating: timit_11/train_11.npy   
  inflating: __MACOSX/timit_11/._train_11.npy  
  inflating: timit_11/test_11.npy    
  inflating: __MACOSX/timit_11/._test_11.npy  
  inflating: timit_11/train_label_11.npy  
  inflating: __MACOSX/timit_11/._train_label_11.npy  
data.zip  __MACOSX  sample_data  timit_11


## Preparing Data
Load the training and testing data from the `.npy` file (NumPy array).

In [ ]:
import numpy as np

print('Loading data ...')

data_root='./timit_11/'
train = np.load(data_root + 'train_11.npy')
train_label = np.load(data_root + 'train_label_11.npy')
test = np.load(data_root + 'test_11.npy')

print('Size of training data: {}'.format(train.shape))
print('Size of testing data: {}'.format(test.shape))

Loading data ...
Size of training data: (1229932, 429)
Size of testing data: (451552, 429)


## Create Dataset

In [ ]:
import torch
from torch.utils.data import Dataset

class TIMITDataset(Dataset):
    def __init__(self, X, y=None):
        self.data = torch.from_numpy(X).float()
        if y is not None:
            y = y.astype(np.int)
            self.label = torch.LongTensor(y)
        else:
            self.label = None

    def __getitem__(self, idx):
        if self.label is not None:
            return self.data[idx], self.label[idx]
        else:
            return self.data[idx]

    def __len__(self):
        return len(self.data)


Split the labeled data into a training set and a validation set, you can modify the variable `VAL_RATIO` to change the ratio of validation data.

In [ ]:
VAL_RATIO = 0.2

percent = int(train.shape[0] * (1 - VAL_RATIO))
train_x, train_y, val_x, val_y = train[:percent], train_label[:percent], train[percent:], train_label[percent:]
print('Size of training set: {}'.format(train_x.shape))
print('Size of validation set: {}'.format(val_x.shape))

Size of training set: (983945, 429)
Size of validation set: (245987, 429)


Create a data loader from the dataset, feel free to tweak the variable `BATCH_SIZE` here.

In [ ]:
BATCH_SIZE = 256

from torch.utils.data import DataLoader

train_set = TIMITDataset(train_x, train_y)
val_set = TIMITDataset(val_x, val_y)
train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle=True) #only shuffle the training data
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=False)

Cleanup the unneeded variables to save memory.<br>

**notes: if you need to use these variables later, then you may remove this block or clean up unneeded variables later<br>the data size is quite huge, so be aware of memory usage in colab**

In [ ]:
import gc

del train, train_label, train_x, train_y, val_x, val_y
gc.collect()

153

## Create Model

Define model architecture, you are encouraged to change and experiment with the model architecture.

In [ ]:
import torch
import torch.nn as nn

class Classifier(nn.Module):
    def __init__(self):
        super(Classifier, self).__init__()
        
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 1024)
        self.layer3 = nn.Linear(1024, 1024)
        self.layer4 = nn.Linear(1024, 512)
        self.layer5 = nn.Linear(512, 256)
        self.layer6 = nn.Linear(256, 128)
        self.layer7 = nn.Linear(128, 64)
        self.out = nn.Linear(64, 39)

        self.layer1_bn = nn.BatchNorm1d(1024)
        self.layer2_bn = nn.BatchNorm1d(1024)
        self.layer3_bn = nn.BatchNorm1d(1024)
        self.layer4_bn = nn.BatchNorm1d(512)
        self.layer5_bn = nn.BatchNorm1d(256)
        self.layer6_bn = nn.BatchNorm1d(128)
        self.layer7_bn = nn.BatchNorm1d(64)

        '''
        self.layer1 = nn.Linear(429, 1024)
        self.layer2 = nn.Linear(1024, 684)
        self.layer3 = nn.Linear(684, 458)
        self.layer4 = nn.Linear(458, 306)
        self.layer5 = nn.Linear(306, 204)
        self.out = nn.Linear(204, 39)

        self.layer1_bn = nn.BatchNorm1d(1024)
        self.layer2_bn = nn.BatchNorm1d(684)
        self.layer3_bn = nn.BatchNorm1d(458)
        self.layer4_bn = nn.BatchNorm1d(306)
        self.layer5_bn = nn.BatchNorm1d(204)
        '''

        self.dropout = nn.Dropout(p=0.2)
        self.dropout_out = nn.Dropout(p=0.5)

        self.act_fn = nn.PReLU()

    def forward(self, x):
        x = self.layer1(x)
        x = self.layer1_bn(x)
        x = self.act_fn(x)

        x = self.dropout(x)

        x = self.layer2(x)
        x = self.layer2_bn(x)
        x = self.act_fn(x)

        x = self.dropout(x)

        x = self.layer3(x)
        x = self.layer3_bn(x)
        x = self.act_fn(x)

        x = self.dropout(x)

        x = self.layer4(x)
        x = self.layer4_bn(x)
        x = self.act_fn(x)

        x = self.dropout(x)

        x = self.layer5(x)
        x = self.layer5_bn(x)
        x = self.act_fn(x)

        x = self.dropout(x)

        x = self.layer6(x)
        x = self.layer6_bn(x)
        x = self.act_fn(x)

        x = self.dropout(x)

        x = self.layer7(x)
        x = self.layer7_bn(x)
        x = self.act_fn(x)

        x = self.out(x)

        x = self.dropout_out(x)
        
        return x

## Training

In [ ]:
#check device
def get_device():
  return 'cuda' if torch.cuda.is_available() else 'cpu'

Fix random seeds for reproducibility.

In [ ]:
# fix random seed
def same_seeds(seed):
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed(seed)
        torch.cuda.manual_seed_all(seed)  
    np.random.seed(seed)  
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True

Feel free to change the training parameters here.

In [ ]:
# fix random seed for reproducibility
same_seeds(0)

# get device 
device = get_device()
print(f'DEVICE: {device}')

# training parameters
num_epoch = 50               # number of training epoch
learning_rate = 0.0001       # learning rate

# the path where checkpoint saved
model_path = './model.ckpt'

# create model, define a loss function, and optimizer
model = Classifier().to(device)
criterion = nn.CrossEntropyLoss() 
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate, weight_decay=0.01)

DEVICE: cuda


In [ ]:
# start training

best_acc = 0.0
for epoch in range(num_epoch):
    train_acc = 0.0
    train_loss = 0.0
    val_acc = 0.0
    val_loss = 0.0

    # training
    model.train() # set the model to training mode
    for i, data in enumerate(train_loader):
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad() 
        outputs = model(inputs) 
        batch_loss = criterion(outputs, labels)
        _, train_pred = torch.max(outputs, 1) # get the index of the class with the highest probability
        batch_loss.backward() 
        optimizer.step() 

        train_acc += (train_pred.cpu() == labels.cpu()).sum().item()
        train_loss += batch_loss.item()

    # validation
    if len(val_set) > 0:
        model.eval() # set the model to evaluation mode
        with torch.no_grad():
            for i, data in enumerate(val_loader):
                inputs, labels = data
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                batch_loss = criterion(outputs, labels) 
                _, val_pred = torch.max(outputs, 1) 
            
                val_acc += (val_pred.cpu() == labels.cpu()).sum().item() # get the index of the class with the highest probability
                val_loss += batch_loss.item()

            print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f} | Val Acc: {:3.6f} loss: {:3.6f}'.format(
                epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader), val_acc/len(val_set), val_loss/len(val_loader)
            ))

            # if the model improves, save a checkpoint at this epoch
            if val_acc > best_acc:
                best_acc = val_acc
                torch.save(model.state_dict(), model_path)
                print('saving model with acc {:.3f}'.format(best_acc/len(val_set)))
    else:
        print('[{:03d}/{:03d}] Train Acc: {:3.6f} Loss: {:3.6f}'.format(
            epoch + 1, num_epoch, train_acc/len(train_set), train_loss/len(train_loader)
        ))

# if not validating, save the last epoch
if len(val_set) == 0:
    torch.save(model.state_dict(), model_path)
    print('saving model at last epoch')


[001/050] Train Acc: 0.322401 Loss: 2.614204 | Val Acc: 0.644156 loss: 1.557705
saving model with acc 0.644
[002/050] Train Acc: 0.365207 Loss: 2.388401 | Val Acc: 0.677430 loss: 1.364989
saving model with acc 0.677
[003/050] Train Acc: 0.379715 Loss: 2.319119 | Val Acc: 0.694159 loss: 1.256639
saving model with acc 0.694
[004/050] Train Acc: 0.387772 Loss: 2.279097 | Val Acc: 0.704362 loss: 1.206130
saving model with acc 0.704
[005/050] Train Acc: 0.393332 Loss: 2.249732 | Val Acc: 0.711334 loss: 1.157824
saving model with acc 0.711
[006/050] Train Acc: 0.398396 Loss: 2.225969 | Val Acc: 0.715985 loss: 1.124929
saving model with acc 0.716
[007/050] Train Acc: 0.401621 Loss: 2.208215 | Val Acc: 0.720957 loss: 1.093023
saving model with acc 0.721
[008/050] Train Acc: 0.406081 Loss: 2.188700 | Val Acc: 0.725103 loss: 1.066696
saving model with acc 0.725
[009/050] Train Acc: 0.409677 Loss: 2.173421 | Val Acc: 0.729396 loss: 1.044962
saving model with acc 0.729
[010/050] Train Acc: 0.41247

## Testing

Create a testing dataset, and load model from the saved checkpoint.

In [ ]:
# create testing dataset
test_set = TIMITDataset(test, None)
test_loader = DataLoader(test_set, batch_size=BATCH_SIZE, shuffle=False)

# create model and load weights from checkpoint
model = Classifier().to(device)
model.load_state_dict(torch.load(model_path))

<All keys matched successfully>

Make prediction.

In [ ]:
predict = []
model.eval() # set the model to evaluation mode
with torch.no_grad():
    for i, data in enumerate(test_loader):
        inputs = data
        inputs = inputs.to(device)
        outputs = model(inputs)
        _, test_pred = torch.max(outputs, 1) # get the index of the class with the highest probability

        for y in test_pred.cpu().numpy():
            predict.append(y)

Write prediction to a CSV file.

After finish running this block, download the file `prediction.csv` from the files section on the left-hand side and submit it to Kaggle.

In [ ]:
with open('prediction.csv', 'w') as f:
    f.write('Id,Class\n')
    for i, y in enumerate(predict):
        f.write('{},{}\n'.format(i, y))